In [1]:
# todo: check the model cfg: augmentations, policy

In [1]:
from tqdm import tqdm
import numpy as np

import torch
from torch import nn
from utils import parse_cfg, iou_vectorized
from darknet import Darknet

In [2]:
darknet = Darknet('cfg/yolov3_test_todel.cfg')

we also can remove bias due to bn
make_layers returns net_info as well. check whether it"s necessary
shortcut is using output[i-1] instead of x check whether works with x
NOTE THAT CONV BEFORE YOLO USES (num_classes filters) * num_anch


In [3]:
x = torch.randn((2, 3, 416, 416))
darknet.forward(x, 'cpu').shape

/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([2, 10647, 85])

In [4]:
weight_file = 'weights/yolov3.weights'
darknet.load_weights(weight_file)

In [5]:
# todo nms
x = torch.randn((2, 3, 416, 416))
device = torch.device('cpu:0')
predictions = darknet.forward(x, device)
predictions[:, :, 4] *= 1000 # to be removed. just for testing
objectness_thres = 0.2
nms_thres = 0.4
classes = 80 # darknet.layers_list[-1][0].classes

/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [6]:
bboxes1 = torch.Tensor([(6, 5, 4, 2), (3, 6, 4, 2), (8, 4, 2, 8), (11, 7, 2, 2), (8, 0, 4, 2)]).float()
bboxes2 = bboxes1[0].unsqueeze(0)

iou_vectorized(bboxes1, bboxes2)

tensor([[1.0000],
        [0.0667],
        [0.0909],
        [0.0000],
        [0.0000]])

In [7]:
# Non-max Suppression:
# 1. Filter out predictions with low objectness score
# 2. 
# objectiveness filtering:
# replace all boxes with predicted probability lower than objectness_thres 
# with zeros: calculate 0/1 mask and apply it to the prediction tensor
# Note: '>' returns Byte but '*' needs Float unsqueezed back to 3D
# objectness_mask = (predictions[:, :, 4] > objectness_thres).float().unsqueeze(2)
# predictions = predictions * objectness_mask

# # 


# image_pred = predictions[0]
# image_pred.shape

In [14]:
print('changing predictions in the nms loop make sure that it is not used later')

detections = [None] * len(predictions)

for i, prediction in enumerate(predictions):
#     print(prediction.shape)
    objectness_mask = (prediction[:, 4] > objectness_thres)#.float().unsqueeze(1)
    prediction = prediction[objectness_mask]
    
    # if no object on an image found, continue with the next image
    if prediction.size(0) == 0:
        continue
        
    print(prediction.shape)
    pred_score, pred_classes = torch.max(prediction[:, 5:5+classes], dim=1, keepdim=True)
#     print(pred_classes.shape, pred_score.shape)
    # detections: (cx, cy, w, h, obj_score, top_class_score, top_class_idx)
    prediction = torch.cat((prediction[:, :5], pred_score.float(), pred_classes.float()), dim=1)
#     print(prediction.shape)
    unique_classes = pred_classes.unique().float()
#     print(unique_classes.shape)
    
    detections_after_nms = []
    
    for cls in tqdm(unique_classes):
        prediction_4_cls = prediction[prediction[:, 6] == cls]
        sort_pred_idxs = torch.sort(prediction_4_cls[:, 4], descending=True)[1]
        prediction_4_cls = prediction_4_cls[sort_pred_idxs]          
        
        while len(prediction_4_cls) > 0:
            detections_after_nms.append(prediction_4_cls[0].unsqueeze(0))
            
            if len(prediction_4_cls) == 1:
                break
            
#             print('prediction_4_cls[0, :5].shape, prediction_4_cls[1:, :5].shape')
#             print(prediction_4_cls[0, :5].shape, prediction_4_cls[1:, :5].shape)
            iou = iou_vectorized(prediction_4_cls[0, :5].unsqueeze(0), prediction_4_cls[1:, :5])
            iou = iou.reshape(-1)
#             print(iou.shape)
#             print('iou < nms_thres')
#             print(iou < nms_thres)
#             print(iou.shape)
            prediction_4_cls = prediction_4_cls[1:][iou < nms_thres]
            
        # append to detections

 54%|█████▍    | 28/52 [00:00<00:00, 156.98it/s]

changing predictions in the nms loop make sure that it is not used later
torch.Size([1420, 85])


 60%|██████    | 33/55 [00:00<00:00, 311.90it/s]

torch.Size([1353, 85])


100%|██████████| 55/55 [00:00<00:00, 140.34it/s]


In [ ]:
from matplotlib import pyplot as plt
import cv2

plt.figure()
img = cv2.imread('dog-cycle-car.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.shape)
img = cv2.resize(img, (416, 416)) / 255
plt.imshow(img)
plt.show()
img = img.transpose((2, 0, 1))
img = torch.from_numpy(img).float()
img = img.unsqueeze(0)
print(img.shape)
pred = darknet.forward(img, 'cpu')
pred

In [7]:
sum(p.numel() for p in darknet.parameters() if p.requires_grad)

AttributeError: 'Darknet' object has no attribute 'numel'

In [22]:
bn.runn.data

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
nn.BatchNorm2d()